In [6]:
import tensorflow as tf
import numpy as np
import pickle

with open('./conv_lstm_sequence_data.pc', 'rb') as f:
    raw_data = pickle.load(f)

print("raw dataset shape: {}".format(raw_data.shape))

raw dataset shape: (17006, 30, 40)


In [14]:
from tensorflow.data import Dataset

seq_len = 18 # it's 1 hour and a half
batch_size = 5

ds = Dataset.from_tensor_slices(raw_data)
sequences = ds.batch(seq_len+1, drop_remainder=True)

ds = sequences.map(lambda x: (x[:-1], x[1:]))
ds = ds.shuffle(10000).batch(batch_size, drop_remainder=True)

In [15]:
ds

<BatchDataset shapes: ((5, 18, 30, 40), (5, 18, 30, 40)), types: (tf.float64, tf.float64)>

In [46]:
from tensorflow.keras.models import Sequential

model = Sequential([
    tf.keras.layers.ConvLSTM2D(filters=20, kernel_size=(3,3), padding='same', input_shape=(None, 30, 40, 1), return_sequences=True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ConvLSTM2D(filters=20, kernel_size=(3,3), padding='same', return_sequences=True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv3D(filters=1, kernel_size=(3,3,3), activation='sigmoid', padding='same')
])

model.compile(loss='mse', optimizer='adadelta')
model.summary()

ValueError: Input 0 of layer conv_lst_m2d_42 is incompatible with the layer: expected ndim=5, found ndim=4. Full shape received: [None, 30, 40, 1]

In [44]:
model.fit(ds)

ValueError: Error when checking input: expected conv_lst_m2d_38_input to have 5 dimensions, but got array with shape (5, 18, 30, 40)